In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from darts import TimeSeries

df = pd.read_csv('../csv/2Jul67_feature.csv')
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])
df.tail()

In [ ]:
df.info()

In [ ]:
from darts.models import (
    LinearRegressionModel,
    RegressionModel,
    LightGBMModel,
    XGBModel,
    CatBoostModel,
)
from darts.metrics import mape, mse, mae

In [ ]:
# make timeseries Object
series = TimeSeries.from_dataframe(df,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count', 'year', 'month', 'day', 'dayofweek_num', 'midmonth', 'payday', 'double_date'],
                                   freq="D")

In [ ]:
print(series)

In [ ]:
target_ts = series[['unique_order_count']]
future_cov_ts = series[['year', 'month', 'day', 'dayofweek_num', 'midmonth', 'payday', 'double_date']]

cut_off = pd.Timestamp("2024-05-01")

target_training, target_validation =target_ts.split_before(cut_off)

target_training.plot()
target_validation.plot()
plt.show()

In [ ]:
# Making a parameter dictionary for use in the gridsearch function
# Gridsearch will return the best model that hasn't been trained yet.
from darts.models import LinearRegressionModel
from darts.metrics import mape, mae, mse, mase
from math import sqrt

parameters = {
    "lags": list(range(1, 36)),
    "lags_future_covariates": [(0,7)],
    "output_chunk_length": [7],
    "random_state": [42]
}

best_parameter = LinearRegressionModel.gridsearch(
    parameters=parameters,
    series = target_ts, #note (training + validation) X only training set!!!
    future_covariates= future_cov_ts,
    start=cut_off,
    forecast_horizon=7,
    stride=1,          
    last_points_only=False,  
    metric=mse,
    verbose=-1
)

In [ ]:
print(best_parameter)
best_model = best_parameter[0]

In [ ]:
best_model.fit(series=target_training, future_covariates=future_cov_ts)

In [ ]:
from darts.models import LinearRegressionModel
from darts.metrics import mape, mae, mse, mase
from math import sqrt

# result from training , validation only
prediction = best_model.predict(len(target_validation))

In [ ]:
mape_score = mape(target_validation, prediction)
print(f'score MAPE: {round(mape_score, 4)}')
print(f'score ACC: {100-(round(mape_score, 4))}')

mse_score = mse(target_validation, prediction)
print(f'score MSE: {round(mse_score, 4)}')

print(f'score RMSE: {round(sqrt(mse_score), 4)}')

mae_score = mae(target_validation, prediction)
print(f'score MAE: {round(mae_score, 4)}')

In [ ]:
plt.figure(figsize=(20, 10))  # ปรับขนาดของภาพ (กว้าง x สูง)
##prediction
prediction.plot(label="prediction")
target_training.plot(label="training")
target_validation.plot(label="validation")

plt.legend()
plt.title('Train, Validation, and Prediction')
plt.xlabel('Date')
plt.ylabel('Order Unit')
plt.show()